<a href="https://colab.research.google.com/github/treyra/CS155_miniproject/blob/data_visualization_v1/neural_net_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/traindata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_train.head()

In [ ]:
df_test = pd.read_csv('https://raw.githubusercontent.com/treyra/CS155_miniproject/data_visualization_v1/data_cleanup/testdata_imputed_normalized_standard.csv',sep=',', header=0,index_col=0)
df_test.head()

In [ ]:
# drop features
df_test = df_test.drop(['FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)
df_train = df_train.drop(['FIPS_CODE', 'YEAR', 'SOURCE_REPORTING_UNIT_NAME','STATE'],axis=1)
df_test.head()

In [ ]:
# use mlp for prediction on multi-label classification # code modified from https://machinelearningmastery.com/multi-label-classification-with-deep-learning/
from numpy import asarray
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from pdb import set_trace as st

# get the model
def get_model(n_inputs, n_outputs):
  # 0.792 in kaggle with dropped features and 40 epochs
  model = Sequential()
  model.add(Dense(200, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(150, input_dim=200, kernel_initializer='he_uniform', activation='relu')) # tried selu also 
  model.add(Dense(100, input_dim=150, kernel_initializer='he_uniform', activation='relu')) # tried selu also 
  # last layer fix for multi-classification probabilities
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam') # tried adamax also
  return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
  with tf.device('/device:GPU:0'):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
      # prepare data
      X_train, X_test = X[train_ix], X[test_ix]
      y_train, y_test = y[train_ix], y[test_ix]
      # define model
      model = get_model(n_inputs, n_outputs)
      # fit model
      model.fit(X_train, y_train, verbose=0, epochs=10)
      # make a prediction on the test set
      yhat = model.predict(X_test)
      # round probabilities to class labels
      #yhat = yhat.round()
      # calculate accuracy
      acc = roc_auc_score(y_test, yhat, multi_class='ovr')
      # store result
      print('>%.3f' % acc)
      results.append(acc)
    return results

# load dataset
df_train_X = df_train.drop(['id','LABEL','0','1','2','3'], axis =1)
df_train_y = df_train.drop(['id','LATITUDE','LONGITUDE','DISCOVERY_TIME','FIRE_SIZE','FIPS_NAME','DAY','LABEL'], axis=1)

X = df_train_X.to_numpy()
y = df_train_y.to_numpy()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model

model = get_model(n_inputs, n_outputs)
# fit the model on all data
with tf.device('/device:GPU:0'):
  # simple early stopping
  es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
  # mc = ModelCheckpoint('best_model.h5', monitor='loss', mode='max', verbose=1, save_best_only=True)
  history = model.fit(X, y, verbose=1, epochs=50, callbacks=[es])
  
# compute accuracy
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))

In [ ]:

# make a prediction for test data
df_test_noid = df_test.drop(['id'], axis=1)
X_test = df_test_noid.to_numpy()
print(X_test.shape)
df_test_noid.head()

y_test = saved_model.predict(X_test)
print('Predicted: %s' % y_test[0])
print(y_test)

In [ ]:
# save as csv_file in sample submission format
# put it together in a dataframe and save
from google.colab import  drive

def save_csv(df, savestr):
  drive.mount("/drive", force_remount=True)
  df.to_csv('/drive/My Drive/CS155_miniproject_1/'+str(savestr)+'.csv')

df = pd.DataFrame(y_test)
predictions = pd.DataFrame(df_test['id'])
predictions['P1']=y_test[:,0]
predictions['P2']=y_test[:,1]
predictions['P3']=y_test[:,2]
predictions['P4']=y_test[:,3]
predictions = predictions.set_index('id')
predictions.head()

In [ ]:
save_csv(predictions,'predictions_neural_net_standard')